In [1]:
import pandas as pd
import numpy as np
from itertools import chain

In [49]:
df = pd.read_csv('prepost_matched_debt_files.csv', index_col = 0)

In [50]:
def stringToList(x):
    lst = x.strip('][\"\'').split(', ') if not pd.isnull(x) else x
    return [x.strip('\"\' ') for x in lst ] if not pd.isnull(x) else x

In [51]:
df['full name'] = df['full name'].apply(lambda x: stringToList(x))
df['Same State Loan Office'] = df['Same State Loan Office'].apply(lambda x: 
                                                                  stringToList(x))
df['Different State Loan Office'] = df['Different State Loan Office'].apply(lambda x: 
                                                                            stringToList(x))
df['Same State Liquidated Debt'] = df['Same State Liquidated Debt'].apply(lambda x: 
                                                                          stringToList(x))
df['Different State Liquidated Debt'] = df['Different State Liquidated Debt'].apply(lambda x: 
                                                                                    stringToList(x))
df['Pierce Certificates'] = df['Pierce Certificates'].apply(lambda x: 
                                                            stringToList(x))

# How many unique individuals were issued 6 percent stocks and deferred 6 percent stocks in 1790 and after?

In [52]:
df.groupby('state')['full name'].count()

state
CT    752
GA     53
MD    369
NC     54
NH    169
NJ    567
NY    868
PA    858
RI    507
SC    270
VA    515
Name: full name, dtype: int64

# How many unique groups of individuals
- were original purchasers of loan office certicates of the same state as the 6 percent stock?
- were original purchasers of loan office certicates issued from another state?
- were original recipients of liquidated debt certicates issued by the same-state loan office? other state loan offices?
- were original recipients of the Pierce Certicates?

### How many unique groups of individuals were original purchasers of loan office certicates of the same state as the 6 percent stock?

In [60]:
df[df['Same State Loan Office'].apply(lambda x: 
                                      type(x) == list)]['state'].value_counts()

PA    328
CT    325
MD    108
VA     85
NY     66
NH     63
NJ     26
SC     14
RI     14
GA      2
Name: state, dtype: int64

### How many unique groups of individuals were original purchasers of loan office certicates of a different state as the 6 percent stock?

In [61]:
df[df['Different State Loan Office'].apply(lambda x: 
                                           type(x) == list)]['state'].value_counts()

PA    136
CT    117
VA    106
MD     65
NY     48
NH     42
NJ     30
SC     18
RI     17
GA      2
NC      1
Name: state, dtype: int64

##### Aside: How many unique groups of individuals were original purchasers of loan office certicates of a different and same state as the 6 percent stock?

In [62]:
df[[type(ss) == list and type(ds) == list for 
    ss, ds in zip(df['Same State Loan Office'], 
                  df['Different State Loan Office'])]]['state'].value_counts()

PA    93
CT    65
VA    41
MD    25
NJ    18
NY    18
NH    14
SC    10
RI     8
GA     2
Name: state, dtype: int64

### How many unique groups of individuals were original recipients of liquidated debt certificates issued by the same state loan office?

In [63]:
df[df['Same State Liquidated Debt'].apply(lambda x: 
                                          type(x) == list)]['state'].value_counts()

PA    228
CT    111
NY     56
NH     29
VA     15
NJ      9
SC      7
RI      5
MD      4
GA      1
Name: state, dtype: int64

### How many unique groups of individuals were original recipients of liquidated debt certificates issued by another state loan office?

In [64]:
df[df['Different State Liquidated Debt'].apply(lambda x: 
                                               type(x) == list)]['state'].value_counts()

CT    188
PA    138
NY     78
VA     61
NH     59
MD     42
NJ     34
RI     21
SC     15
GA      1
Name: state, dtype: int64

##### Aside: How many unique groups of individuals were original purchasers of liquidated debt certificates from the same and another loan office?

In [65]:
df[[type(ss) == list and type(ds) == list for 
    ss, ds in zip(df['Same State Liquidated Debt'], 
                  df['Different State Liquidated Debt'])]]['state'].value_counts()

CT    110
PA     69
NY     52
NH     29
VA     13
NJ      8
SC      5
RI      5
MD      4
GA      1
Name: state, dtype: int64

### How many unique groups of individuals were original recipients of the Pierce Certicates, corresponding to the same state?

In [66]:
df[df['Pierce Certificates'].apply(lambda x: 
                                   type(x) == list)]['state'].value_counts()

CT    182
PA    172
VA    139
MD     85
NY     60
NH     35
NJ     23
SC     16
RI     15
NC      2
GA      1
Name: state, dtype: int64

### For how many unique groups of individuals do the face values of their pre-1790 certificates equal the sum of the face values of the 6 percent and deferred 6 percent stocks that they received?

##### Number of people with non-zero assets and that the debt values from pre-1790 and post-1790 are equal

In [73]:
df[[abs(tt-dt)<=1 and tt != 0 
    for tt, dt in zip(df['Total'], df['Debt Total'])]]

,full name,state,Same State Loan Office,Different State Loan Office,Same State Liquidated Debt,Different State Liquidated Debt,Pierce Certificates,Total,SSLO Total,DSLO Total,SSLD Total,DSLD Total,PC Total,Debt Total,tot_pre1790_certs
60,[Aaron Kelsey],CT,[Aaron Kelsey],NaN,NaN,NaN,NaN,95.10,95.10834,0.0,0.000000,0.0,0.0,95.108340,1
73,[Abel Morse],CT,[Abel Morse],NaN,NaN,NaN,NaN,1388.83,1388.86930,0.0,0.000000,0.0,0.0,1388.869300,1
74,[Abel Patchen],CT,[Abel Patchen],NaN,NaN,NaN,NaN,153.96,153.97916,0.0,0.000000,0.0,0.0,153.979160,1
102,[Abner Corson],PA,[Abner Corson],NaN,NaN,NaN,NaN,60.23,60.23889,0.0,0.000000,0.0,0.0,60.238890,1
112,[Abraham Brinsmade],CT,[Abraham Brinsmade],NaN,NaN,NaN,NaN,400.00,400.00000,0.0,0.000000,0.0,0.0,400.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4713,[William Dickensheet],MD,[William Dickensheet],NaN,NaN,NaN,NaN,50.00,50.00833,0.0,0.000000,0.0,0.0,50.008330,1
4783,[William House],MD,[William House],NaN,NaN,NaN,NaN,25.47,25.48750,0.0,0.000000,0.0,0.0,25.487500,1
4890,[William Sitgreaves],PA,NaN,NaN,[William Sitgreaves],NaN,NaN,1277.57,0.00000,0.0,1277.577778,0.0,0.0,1277.577778,1
4958,[William Wyvell],MD,[William Wyvell],NaN,NaN,NaN,NaN,22.50,22.50000,0.0,0.000000,0.0,0.0,22.500000,1


In [74]:
df[df['tot_pre1790_certs'] != 0].shape[0]

1736

In [75]:
df[[abs(tt-dt)<=1 and tt != 0 
    for tt, dt in zip(df['Total'], df['Debt Total'])]].shape[0]

94

##### State Distribution

In [76]:
df[[abs(tt-dt)<=1 and tt != 0 
    for tt, dt in zip(df['Total'], df['Debt Total'])]]['state'].value_counts()

CT    48
MD    18
PA    17
NH     6
VA     5
Name: state, dtype: int64

##### Number of people with zero assets and equal values
Most of these occur when someone has 3% but no 6% certificates

In [77]:
df[[tt == dt and tt == 0 
    for tt, dt in zip(df['Total'], df['Debt Total'])]].shape[0]

1283

##### State Distribution

In [78]:
df[[tt == dt and tt == 0 
    for tt, dt in zip(df['Total'], df['Debt Total'])]]['state'].value_counts()

NY    669
NJ    526
VA     24
CT     23
RI     19
MD     10
PA      9
SC      3
Name: state, dtype: int64

### Who, and how many unique groups of people redeemed certificates with Hamilton and were not original owners/purchasers

In [79]:
df[[tot == 0 and tt != 0 for tot, tt in zip(df['tot_pre1790_certs'], df['Total'])]].shape[0]

1984

In [80]:
df[[tot == 0 and tt != 0 for tot, tt in zip(df['tot_pre1790_certs'], df['Total'])]]

,full name,state,Same State Loan Office,Different State Loan Office,Same State Liquidated Debt,Different State Liquidated Debt,Pierce Certificates,Total,SSLO Total,DSLO Total,SSLD Total,DSLD Total,PC Total,Debt Total,tot_pre1790_certs
4,[James O'Hara],PA,NaN,NaN,NaN,NaN,NaN,33.44,0.0,0.0,0.0,0.0,0.0,0.0,0
7,[Agness ann],PA,NaN,NaN,NaN,NaN,NaN,883.63,0.0,0.0,0.0,0.0,0.0,0.0,0
8,[Anne Maria],PA,NaN,NaN,NaN,NaN,NaN,255.77,0.0,0.0,0.0,0.0,0.0,0.0,0
9,[Antoine Rene Charter Mathurin de la Forest],PA,NaN,NaN,NaN,NaN,NaN,6771.75,0.0,0.0,0.0,0.0,0.0,0.0,0
10,[Charles Godfreid],PA,NaN,NaN,NaN,NaN,NaN,3260.83,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,[Zebulon Waterman],CT,NaN,NaN,NaN,NaN,NaN,306.61,0.0,0.0,0.0,0.0,0.0,0.0,0
4977,[Zephaniah Andrews],RI,NaN,NaN,NaN,NaN,NaN,1728.74,0.0,0.0,0.0,0.0,0.0,0.0,0
4978,[Zephaniah Brown],RI,NaN,NaN,NaN,NaN,NaN,2415.08,0.0,0.0,0.0,0.0,0.0,0.0,0
4979,[Zephaniah Davis],CT,NaN,NaN,NaN,NaN,NaN,77.20,0.0,0.0,0.0,0.0,0.0,0.0,0


##### state distribution

In [81]:
df[[tot == 0 and tt != 0 for tot, tt in zip(df['tot_pre1790_certs'], df['Total'])]]['state'].value_counts()

RI    459
PA    344
VA    275
CT    246
SC    242
MD    178
NY     83
NH     55
GA     51
NC     51
Name: state, dtype: int64

In [82]:
namelsts = df[[tot == 0 and tt != 0 for 
               tot, tt in zip(df['tot_pre1790_certs'], df['Total'])]]['full name'].apply(lambda x: pd.Series(x)).values
names = [x for x in list(chain.from_iterable(namelsts)) if not pd.isnull(x)]

In [83]:
name_state_asset = pd.DataFrame({}, columns = ['Full Name', 'Total', 'state'])

In [85]:
name_df = df[[tot == 0 and tt != 0 for 
              tot, tt in zip(df['tot_pre1790_certs'], df['Total'])]]['full name'].apply(lambda x: pd.Series(x))
name_df
for name in names:
    ind1 = name_df[name_df[0] == name].index.tolist()
    ind2 = name_df[name_df[1] == name].index.tolist()
    ind3 = name_df[name_df[2] == name].index.tolist()
    ind4 = name_df[name_df[3] == name].index.tolist()
    ind5 = name_df[name_df[4] == name].index.tolist()
    ind1.extend(ind2)
    ind1.extend(ind3)
    ind1.extend(ind4)
    ind1.extend(ind5)
    ind1 = list(set(ind1))
    tot_assets = df.loc[ind1, 'Total'].sum()
    state = df.loc[ind1, 'state'].values[0]
    add_df = pd.DataFrame()
    add_df['Full Name'] = [name]
    add_df['Total'] = [tot_assets]
    add_df['state'] = [state]
    name_state_asset = pd.concat([name_state_asset, add_df])

In [86]:
cd_names = pd.DataFrame(name_state_asset.groupby(['Full Name', 'state'])['Total'].sum()).reset_index()

In [88]:
### How many unique people redeemed certificates with Hamilton and were not original owners/purchasers

In [89]:
len(set(cd_names['Full Name']))

1979

In [92]:
cd_names.to_csv("speculator_names.csv")